In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys

sys.path.append('/content/gdrive/My Drive/Colab Notebooks/ECE_Project')

In [0]:
import RNN_Lib
import numpy as np 
import torch
from torch.autograd import Variable
import random
import pdb

#cuda = torch.device("cuda:0") #default cuda

In [21]:
dir = '/content/gdrive/My Drive/Colab Notebooks/ECE_Project'
X_test = np.load(dir + '/X_test.npy')
y_test = np.load(dir + '/y_test.npy')
person_train_valid = np.load(dir + '/person_train_valid.npy') #tells us which subject the indices of xtest correspond to 
X_train_valid = np.load(dir + '/X_train_valid.npy')
y_train_valid = np.load(dir + '/y_train_valid.npy')
person_test = np.load(dir + '/person_test.npy')  #tells us which subject the indices of xtrain correspond to 
print ('Training/Valid data shape: {}' .format(X_train_valid.shape))
print ('Test data shape: {}' .format(X_test.shape))
print ('Training/Valid target shape: {}' .format(y_train_valid.shape))
print ('Test target shape: {}' .format(y_test.shape))
print ('Person train/valid shape: {}' .format(person_train_valid.shape))
print ('Person test shape: {}' .format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
#transform data into torch-readable data types
#2115 number of trials from 9 people
#four possible classes of outputs
#Xtrain = Variable(torch.from_numpy(X_train_valid)).cuda()
#ytrain = Variable(torch.from_numpy(y_train_valid)).cuda()
#Xtest = Variable(torch.from_numpy(X_test)).cuda()
#ytest = Variable(torch.from_numpy(y_test)).cuda()

Xtrain = torch.from_numpy(X_train_valid)
ytrain = torch.from_numpy(y_train_valid)
Xtest = torch.from_numpy(X_test)
ytest = torch.from_numpy(y_test)

In [0]:
def map_to_class(input_labels):
  mask1 = (input_labels == 769)*0
  mask2 = (input_labels == 770)*1
  mask3 = (input_labels == 771)*2
  mask4 = (input_labels == 772)*3

  return (mask1 + mask2 + mask3 + mask4)

In [27]:
########## INITIALIZE RNN MODEL ########## 

Trials, InputDim, SeqDim = X_train_valid.shape
num_features = InputDim
Tests = y_test.shape


#-----tweak here----#
batch_size = 10
num_layers = 8
hidden_dim = 500
num_steps = 0 #do we need this?
num_class = 4 #4 classes
non_linearity='tanh'
initialization='xavierNorm'
dropout=0
learning_rate = 0.001
L2 = 0.15


#rnn element
rnn_model = RNN_Lib.EEG_RNN(num_features, hidden_dim, num_layers, 0, num_class, non_linearity, initialization, dropout)
rnn_model.to('cuda:0') #--- activate the GPU

#fully connected net
#FC = torch.nn.Linear(hidden_dim, num_class).to('cuda:0')  

DONE INITIALIZING


EEG_RNN(
  (RNN): RNN(22, 500, num_layers=8, batch_first=True)
  (FC): Linear(in_features=500, out_features=4, bias=True)
)

In [0]:
########## CHOOSE LOSS  ###########
loss = torch.nn.CrossEntropyLoss()

######### CHOOSE OPTIMIZER ########
optim = torch.optim.Adam(rnn_model.parameters(), lr=learning_rate, weight_decay=L2)


In [29]:
######## TRAIN IT ########
epochs = 1
iterations = 150
iter_ = 0
for epoch in range(epochs):
  ###DEBUG
  #pdb.set_trace()

  train_correct = 0
  train_total = 0

  for i in range(iterations):
    #put the model in training mode
    rnn_model.train(True)
    #FC.train(True)

    #process the input data
    #idx = np.random.randint(0, Trials, size=batch_size)
    idx = range(0,10)
    xtrain = Xtrain[idx].view(-1, SeqDim, InputDim)
    xtrain = xtrain.to('cuda:0').requires_grad_()

    #forward pass
    optim.zero_grad()
    outFC, hidden = rnn_model.forward(xtrain.float())

    #outFC = FC(outRNN[:,-1, :])

    #values, predicted = torch.max(outFC, 1)

    #map to classes?
    classes = map_to_class(ytrain[idx].long())
    classes = classes.to('cuda:0')

    #backward pass (gradient calculation)
    probs = loss(outFC, classes)
    probs.backward()

    #update weights
    optim.step()

    #update iter counter
    iter_+=1

    #calculate training accuracy
    values, train_pred = torch.max(outFC, 1)
    train_correct += (1*(train_pred == classes)).sum()
    train_total +=float(classes.size(0))

    if (iter_ % 10) == 0:
      rnn_model.eval()
      #FC.eval()
      total = 0
      correct = 0
      for j in range(X_test.shape[0]):
        #process the input data
        #idx = np.random.randint(0, Tests, size=batch_size)
        xtest = Xtest[idx].view(-1, SeqDim, InputDim)
        xtest = xtest.cuda()

        #forward prop
        predict, _ = rnn_model(xtest.float())

        #print(f"size of out: {out.shape}")
#        FC = torch.nn.Linear(hidden_dim, num_class)   
        #predict = FC(out[:,-1, :])
        
        values, predicted_classes = torch.max(predict, 1)
        #print(predicted_classes)
        #print(map_to_class(ytest[idx].long()))
        expected_classes = map_to_class(ytest[idx].long())
        correct +=(1*(predicted_classes == expected_classes.to('cuda:0'))).sum()
        total +=float(ytest[idx].size(0))

      print(f"\ncorrect: {correct}, total: {total}")
      #print(f"prediction: {predicted_classes}\nexpected_classes: {expected_classes}")
      print(f"Iteration: {iter_}, Loss: {probs.item()}, Validation Accuracy: {100*(correct/total)}%, Training Accuracy: {100*(train_correct/train_total)}%")  


correct: 1772, total: 4430.0
Iteration: 10, Loss: 0.5480607748031616, Validation Accuracy: 40.0%, Training Accuracy: 57.0%

correct: 886, total: 4430.0
Iteration: 20, Loss: 0.20085182785987854, Validation Accuracy: 20.0%, Training Accuracy: 73.49999237060547%

correct: 886, total: 4430.0
Iteration: 30, Loss: 0.132486492395401, Validation Accuracy: 20.0%, Training Accuracy: 82.33333587646484%

correct: 886, total: 4430.0
Iteration: 40, Loss: 0.07141055911779404, Validation Accuracy: 20.0%, Training Accuracy: 86.75%

correct: 886, total: 4430.0
Iteration: 50, Loss: 0.057924725115299225, Validation Accuracy: 20.0%, Training Accuracy: 89.40000915527344%

correct: 886, total: 4430.0
Iteration: 60, Loss: 0.05407357215881348, Validation Accuracy: 20.0%, Training Accuracy: 91.16667175292969%

correct: 886, total: 4430.0
Iteration: 70, Loss: 0.05320935323834419, Validation Accuracy: 20.0%, Training Accuracy: 92.42857360839844%

correct: 886, total: 4430.0
Iteration: 80, Loss: 0.052110768854618

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])